<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NRMS: Neural News Recommendation with Multi-Head Self-Attention
NRMS \[1\] is a neural news recommendation approach with multi-head selfattention. The core of NRMS is a news encoder and a user encoder. In the newsencoder, a multi-head self-attentions is used to learn news representations from news titles by modeling the interactions between words. In the user encoder, we learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. Besides, we apply additive
attention to learn more informative news and user representations by selecting important words and news.

## Properties of NRMS:
- NRMS is a content-based neural news recommendation approach.
- It uses multi-self attention to learn news representations by modeling the iteractions between words and learn user representations by capturing the relationship between user browsed news.
- NRMS uses additive attentions to learn informative news and user representations by selecting important words and news.

## Data format:

### train data
One simple example: <br>

`1 0 0 0 0 Impression:0 User:2903 CandidateNews0:27006,11901,21668,9856,16156,21390,1741,2003,16983,8164 CandidateNews1:8377,10423,9960,5485,20494,7553,1251,17232,4745,9178 CandidateNews2:1607,26414,25830,16156,15337,16461,4004,6230,17841,10704 CandidateNews3:17323,20324,27855,16156,2934,14673,551,0,0,0 CandidateNews4:7172,3596,25442,21596,26195,4745,17988,16461,1741,76 ClickedNews0:11362,8205,22501,9349,12911,20324,1238,11362,26422,19185 ...`
<br>

In general, each line in data file represents one positive instance and n negative instances in a same impression. The format is like: <br>

`[label0] ... [labeln] [Impression:i] [User:u] [CandidateNews0:w1,w2,w3,...] ... [CandidateNewsn:w1,w2,w3,...] [ClickedNews0:w1,w2,w3,...] ...`

<br>

It contains several parts seperated by space, i.e. label part, Impression part `<impresison id>`, User part `<user id>`, CandidateNews part, ClickedHistory part. CandidateNews part describes the target news article we are going to score in this instance, it is represented by (aligned) title words. To take a quick example, a news title may be : `Trump to deliver State of the Union address next week` , then the title words value may be `CandidateNewsi:34,45,334,23,12,987,3456,111,456,432`. <br>
ClickedNewsk describe the k-th news article the user ever clicked and the format is the same as candidate news. Words are aligned in news title. We use a fixed length to describe an article, if the title is less than the fixed length, just pad it with zeros.

### test data
One simple example: <br>
`1 Impression:0 User:6446 CandidateNews0:18707,23848,13490,10948,21385,11606,1251,16591,827,28081 ClickedNews0:27838,7376,16567,28518,119,21248,7598,9349,20324,9349 ClickedNews1:7969,9783,1741,2549,27104,14669,14777,21343,7667,20324 ...`
<br>

In general, each line in data file represents one instance. The format is like: <br>

`[label] [Impression:i] [User:u] [CandidateNews0:w1,w2,w3,...] [ClickedNews0:w1,w2,w3,...] ...`
<br>

## Global settings and imports

In [3]:
import sys
sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources 
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.nrms import NRMSModel
from reco_utils.recommender.newsrec.IO.news_iterator import NewsIterator
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

tmpdir = TemporaryDirectory()

System version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]
Tensorflow version: 1.12.0


## Download and load data

In [4]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'nrms.yaml')
train_file = os.path.join(data_path, r'train.txt')
valid_file = os.path.join(data_path, r'test.txt')
wordEmb_file = os.path.join(data_path, r'embedding.npy')
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/newsrec/', data_path, 'nrms.zip')

100%|██████████| 21.2k/21.2k [00:01<00:00, 12.0kKB/s]


## Create hyper-parameters

In [5]:
epoch=10

In [6]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, epochs=epoch)
print(hparams)

[('attention_hidden_dim', 200), ('batch_size', 64), ('body_size', None), ('cnn_activation', None), ('data_format', 'news'), ('dense_activation', None), ('doc_size', 10), ('dropout', 0.2), ('epochs', 10), ('filter_num', 200), ('gru_unit', 400), ('head_dim', 50), ('head_num', 3), ('his_size', 50), ('iterator_type', None), ('learning_rate', 0.0001), ('loss', 'cross_entropy_loss'), ('metrics', ['group_auc', 'mean_mrr', 'ndcg@5;10']), ('npratio', 4), ('optimizer', 'adam'), ('show_step', 100000), ('subvert_emb_dim', 100), ('subvert_num', None), ('title_size', None), ('type', 'ini'), ('user_emb_dim', 50), ('user_num', 10338), ('vert_emb_dim', 100), ('vert_num', None), ('window_size', 3), ('wordEmb_file', '/tmp/tmp1yw8c6d7/embedding.npy'), ('word_emb_dim', 100), ('word_size', 28929)]


In [7]:
iterator = NewsIterator

## Train the NRMS model

In [8]:
model = NRMSModel(hparams, iterator)

In [9]:
print(model.run_eval(valid_file))

{'group_auc': 0.504, 'mean_mrr': 0.1627, 'ndcg@5': 0.1537, 'ndcg@10': 0.2173}


In [10]:
model.fit(train_file, valid_file)

/data/anaconda/envs/reco_base/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


at epoch 1
train info: logloss loss:1.6053238148591957
eval info: group_auc:0.544, mean_mrr:0.1663, ndcg@10:0.2265, ndcg@5:0.1605
at epoch 1 , train time: 12.3 eval time: 8.4
at epoch 2
train info: logloss loss:1.5829095952364862
eval info: group_auc:0.5467, mean_mrr:0.1643, ndcg@10:0.2286, ndcg@5:0.1529
at epoch 2 , train time: 9.6 eval time: 8.5
at epoch 3
train info: logloss loss:1.5629846645861256
eval info: group_auc:0.5551, mean_mrr:0.1677, ndcg@10:0.2319, ndcg@5:0.1589
at epoch 3 , train time: 9.7 eval time: 8.4
at epoch 4
train info: logloss loss:1.5421645193683857
eval info: group_auc:0.558, mean_mrr:0.168, ndcg@10:0.2287, ndcg@5:0.1573
at epoch 4 , train time: 9.7 eval time: 8.3
at epoch 5
train info: logloss loss:1.5166382327371715
eval info: group_auc:0.5638, mean_mrr:0.1788, ndcg@10:0.2387, ndcg@5:0.17
at epoch 5 , train time: 9.7 eval time: 8.5
at epoch 6
train info: logloss loss:1.4905069336599233
eval info: group_auc:0.5651, mean_mrr:0.1786, ndcg@10:0.2411, ndcg@5:0.172

In [11]:
print(model.run_eval(valid_file))

{'group_auc': 0.5691, 'mean_mrr': 0.1802, 'ndcg@5': 0.1787, 'ndcg@10': 0.2464}


## Reference
\[1\] Wu et al. "Neural News Recommendation with Multi-Head Self-Attention." in Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)<br>